# Charges analysis

Looking at how many times different charges come up. The raw data accessed was pulled from our database, then saved into `data_raw` folder as `offenses-YYYY-MM-DD_v.csv`.

## Goals of this analysis
- See which offenses appear the most
- Isolate and count the cases where the only charge is in a short list including 'Evading arrest', 'Resisting arrest', 'Interfering with police duties' and perhaps a few others.


In [1]:
import agate
import re
import warnings
warnings.filterwarnings('ignore')

## Import the files we need

In [2]:
# This is the data of each decedant and the offense they have
offense_file = '../data_raw/offenses_latest.csv'
# This is a list of all the possible offenses
offense_table_file = '../data_raw/offense_table.csv'
# This is the death tables, after it is run through the File processing notebook
deaths_file = '../exports/deaths_latest.csv'

### Import deaths data

In [3]:
# sets data types on fields agate got wrong
specified_data_types = {
    'tracked_cause': agate.Text(),
    'offense': agate.Text(),
    'case_study': agate.Text(),
    'official_discipline': agate.Text()
}

# this pulls the deaths file that is exported in the File processing notebook
deaths = agate.Table.from_csv(deaths_file, column_types=specified_data_types)

print(deaths)

| column               | data_type |
| -------------------- | --------- |
| id                   | Number    |
| ag_report_url        | Text      |
| first_name           | Text      |
| middle_name          | Text      |
| last_name            | Text      |
| suffix               | Text      |
| slug                 | Text      |
| race                 | Text      |
| gender               | Text      |
| date_of_birth        | Date      |
| date_of_death        | Date      |
| age                  | Number    |
| agency               | Number    |
| restrained           | Boolean   |
| tazed                | Boolean   |
| times_tazed          | Number    |
| pepper_sprayed       | Boolean   |
| official_discipline  | Text      |
| grand_jury_result    | Text      |
| mental_health_issues | Boolean   |
| manner_of_death      | Text      |
| drug_intoxication    | Boolean   |
| cause_of_death       | Text      |
| tracked_cause        | Text      |
| offense              | Text      |
|

### Import offense table

This is the table of the actual offenses and their ID.

In [4]:
# read in the offenses data
offenses = agate.Table.from_csv(offense_file)

# print the column names
print(offenses)

| column    | data_type |
| --------- | --------- |
| id        | Number    |
| Full Name | Text      |
| race      | Text      |
| offense   | Text      |



In [5]:
# print first 5 rows to see what we are looking at
offenses.limit(5).print_table(max_column_width=80)

|  id | Full Name                  | race | offense                                                |
| --- | -------------------------- | ---- | ------------------------------------------------------ |
| 612 | Anthony Timpa              | w    | APOWW/mental health hold                               |
| 423 | Juan Carlos Reyes-Gallardo | h    | Public intoxication                                    |
| 528 | Micheal Olveda             | h    | Outstanding warrants                                   |
| 613 | Manuel Delacruz            | h    | Taking or attempting to take weapon from peace officer |
| 424 | Mario Evans                | b    | Public intoxication                                    |


### Import offenses data

In [6]:
# read in the list of offense choices
offenses_table = agate.Table.from_csv(offense_table_file)

# print the column names
print(offenses_table)

| column  | data_type |
| ------- | --------- |
| id      | Number    |
| offense | Text      |



## Offenses that appear the most

In [7]:
# pivot offenses table and order by most
offenses_pivot = offenses.pivot('offense').order_by('Count', reverse=True)

# print offenses list
offenses_pivot.print_table(max_column_width=None, max_rows=None)

| offense                                                    | Count |
| ---------------------------------------------------------- | ----- |
| Resisting arrest                                           |    59 |
| Public intoxication                                        |    54 |
| Assault on public servant                                  |    38 |
| Evading arrest                                             |    32 |
| Possession of a controlled substance                       |    31 |
| APOWW/mental health hold                                   |    22 |
| Burglary of a habitation                                   |    14 |
| Criminal mischief                                          |    13 |
| Outstanding warrants                                       |    12 |
| None listed                                                |    11 |
| DWI                                                        |    10 |
| Criminal trespass                                          |    10 |
| Aggr

In [8]:
# print the list of offenses and their ID
# The IDs are used to build the minor_offenses
offenses_table.order_by('id').print_table(max_rows=None, max_column_width=None)

| id | offense                                                    |
| -- | ---------------------------------------------------------- |
|  1 | Theft $50-$500                                             |
|  2 | Aggravated assault with a deadly weapon                    |
|  3 | Evading arrest                                             |
|  5 | Criminal mischief                                          |
|  6 | Assault on public servant                                  |
|  7 | DWI                                                        |
|  8 | Resisting arrest                                           |
|  9 | Public intoxication                                        |
| 10 | Aggravated robbery                                         |
| 12 | Assault family violence                                    |
| 13 | Burglary of a habitation                                   |
| 14 | Criminal trespass                                          |
| 15 | Disorderly conduct                       

### Set the list that constitute 'suspect' offenses

The list of offenses we will consider minor for this analysis:

- 3: Evading arrest
- 5: Criminal mischief
- 6: Assault on public servant  
- 8: Resisting arrest
- 9: Public intoxication
- 14: Criminal trespass
- 15: Disorderly conduct
- 19: Driving without a license
- 26: Traffic violations
- 27: Seatbelt violation
- 28: Tampering with evidence
- 30: APOWW/mental health hold
- 34: False identification
- 36: Interfering w/ police duties
- 37: None listed
- 42: Possession of drug paraphernalia
- 51: Failure to stop and give information


In [9]:
# This set the list to be evaluated later
# in the suspect_offense_check function
suspect_offenses = ['3','5','6','8','9','14','15','19','26','27',
                    '28','30','34','36','37','42','45','51',''] #leave last item blank

infractions_story_list = ['3', '8', '9', '17', '42', '48', '30', '37']

resist_arrest_only = ['3','8']

public_intox = ['9']

mental_health = ['30']

traffic_violations = ['19', '27', '26']

drug_possession = ['17', '42', '48']

interference = ['36']

none_listed = ['37']

In [10]:
# We'll select just the columns we need from deaths.
deaths_offenses = deaths.select(['first_name', 'middle_name', 'last_name', 'offense']).order_by('last_name')

# and we pee at those columns:
print(deaths_offenses)

| column      | data_type |
| ----------- | --------- |
| first_name  | Text      |
| middle_name | Text      |
| last_name   | Text      |
| offense     | Text      |



In [11]:
# Printing the whole table here so we can compare
# against our filtered table later
# also, there should be no blanks for offense
deaths_offenses.print_table(max_rows=None)

| first_name  | middle_name   | last_name        | offense  |
| ----------- | ------------- | ---------------- | -------- |
| Pierre      | Tourell       | Abernathy        | 7,3,8    |
| Jesse       |               | Aguirre          | 5,35     |
| Sam         |               | Akin             | 12       |
| Timothy     |               | Alfaro           | 14,9     |
| Steven      | Lee           | Allen            | 48       |
| Raymond     | Luther        | Allen            | 9,8      |
| Jamail      | Joseph        | Amron            | 37       |
| Ross        | Allen         | Anthony          | 30       |
| Corey       | Dean          | Bailey           | 7        |
| Weldon      | Davis         | Baker            | 5        |
| Manuel      | A.            | Baltazar         | 6        |
| Willie      | Ray           | Banks            | 6,9      |
| Herman      |               | Barnes           | 37       |
| Osbaldo     | Xavier        | Barrera          | 64,65    |
| Alfred

| Shirley     | Ann           | South            | 38       |
| Troy        | Marcus        | Stewart          | 30       |
| Lorenzo     | Charles       | Stiggers         | 9        |
| Teddy       |               | Stranghoener     | 30       |
| Lesa        | Ann           | Surratt          | 17       |
| Elcide      | Gabriel       | Sylve            | 9        |
| Daniel      | Rivera        | Tamez            | 3,57,8   |
| John        | Steven        | Thomas           | 5,9,8    |
| Kinzy       | Laquinte      | Thornton         | 6        |
| Anthony     |               | Timpa            | 30       |
| Thuan       |               | Tran             |          |
| Aladino     |               | Trejo            | 9        |
| Alisha      |               | Trevino          | 39       |
| Jeffery     | Garlin        | Trotter          | 10       |
| Adren       | Maurice       | Turner           | 7        |
| Jamaal      | Ray           | Valentine        | 9        |
| Noe   

### Suspect charges function

Sometimes there are cases where charges were very minor to begin with, or charges were the result of the police interaction, and would not have happened otherwise. This process builds a list of cases to investigate further.

The function evaluates the offenses in a given row:
- If the offense field is blank, we keep it. (There shouldn't be ... those cases should have the "None listed" option.)
- If the offense field has an offense listed that is NOT in the suspect offenses list, it is skipped
- If what you are left with is just suspect offenses or blank, we keep it

We store all that in a new table and then print it.

In [12]:
# This is ONLY OFFENSES evaluation function
# It looks at the list of offenses in a row and then compares it
# against offense_list, which is a list passed into the function.
# It returns only rows where charges match the offense_list.
# if there are any others, the row is rejected

def offense_check(row, offense_list):
    if row is None:
        return True
    myList = row.split(',')
    for item in myList:
        if item not in offense_list:
            return False
    return True

In [13]:
# this filters our death_offenses table based on the 
# minor_offense_check function above, and then creates
# a new table of just those rows that pass
suspect_offense_cases = deaths_offenses.where(lambda row: offense_check(row['offense'], suspect_offenses))
print('Records with only "suspect" offenses: {} or {:.1%}\n'.format(
    len(suspect_offense_cases),
    len(suspect_offense_cases) / len(deaths_offenses)
    ))
# prints the deaths with just minor offenses
suspect_offense_cases.print_table(max_rows=None)

Records with only "suspect" offenses: 159 or 55.0%

| first_name  | middle_name   | last_name        | offense |
| ----------- | ------------- | ---------------- | ------- |
| Timothy     |               | Alfaro           | 14,9    |
| Raymond     | Luther        | Allen            | 9,8     |
| Jamail      | Joseph        | Amron            | 37      |
| Ross        | Allen         | Anthony          | 30      |
| Weldon      | Davis         | Baker            | 5       |
| Manuel      | A.            | Baltazar         | 6       |
| Willie      | Ray           | Banks            | 6,9     |
| Herman      |               | Barnes           | 37      |
| John        | Paul          | Bazan            | 6,8     |
| Frank       | Timothy       | Benavides        | 30      |
| Magnolia    | Lucille       | Blakeman         | 9       |
| Charles     | Edward        | Boll             | 15      |
| Anthony     | George        | Brown            | 6       |
| Robert      |               | B

### List number of cases in specific groupings

The groupings are set above where we list out all the offense IDs.

In [14]:
suspect_offense_cases = deaths_offenses.where(lambda row: offense_check(row['offense'], suspect_offenses))
print('Records with only "suspect" offenses: {} or {:.1%}\n'.format(
    len(suspect_offense_cases),
    len(suspect_offense_cases) / len(deaths_offenses)
    ))

infraction_story_cases = deaths_offenses.where(lambda row: offense_check(row['offense'], infractions_story_list))
print('Records for infraction story offenses: {} or {:.1%}\n'.format(
    len(infraction_story_cases),
    len(infraction_story_cases) / len(deaths_offenses)
    ))

resist_evade_cases = deaths_offenses.where(lambda row: offense_check(row['offense'], resist_arrest_only))
print('Records with only Resisting or Evading Arrest charges: {} for {:.1%}\n'.format(
        len(resist_evade_cases),
        len(resist_evade_cases)/ len(deaths_offenses)
    ))

public_intox_cases = deaths_offenses.where(lambda row: offense_check(row['offense'], public_intox))
print('Records with only Public Intoxication charge: {} for {:.1%}\n'.format(
        len(public_intox_cases),
        len(public_intox_cases)/ len(deaths_offenses)
    ))

mental_health_cases = deaths_offenses.where(lambda row: offense_check(row['offense'], mental_health))
print('Records with only Mental Health charge: {} for {:.1%}\n'.format(
        len(mental_health_cases),
        len(mental_health_cases)/ len(deaths_offenses)
    ))

traffic_violations_cases = deaths_offenses.where(lambda row: offense_check(row['offense'], traffic_violations))
print('Records with only Traffic Violation charges: {} for {:.1%}\n'.format(
        len(traffic_violations_cases),
        len(traffic_violations_cases)/ len(deaths_offenses)
    ))

drug_possession_cases = deaths_offenses.where(lambda row: offense_check(row['offense'], drug_possession))
print('Records with only Drug Possession charges: {} for {:.1%}\n'.format(
        len(drug_possession_cases),
        len(drug_possession_cases)/ len(deaths_offenses)
    ))

interference_cases = deaths_offenses.where(lambda row: offense_check(row['offense'], interference))
print('Records with only Interference charges: {} for {:.1%}\n'.format(
        len(interference_cases),
        len(interference_cases)/ len(deaths_offenses)
    ))

none_cases = deaths_offenses.where(lambda row: offense_check(row['offense'], none_listed))
print('Records with only "None listed" charges: {} for {:.1%}\n'.format(
        len(none_cases),
        len(none_cases)/ len(deaths_offenses)
    ))


Records with only "suspect" offenses: 159 or 55.0%

Records for infraction story offenses: 120 or 41.5%

Records with only Resisting or Evading Arrest charges: 24 for 8.3%

Records with only Public Intoxication charge: 36 for 12.5%

Records with only Mental Health charge: 25 for 8.7%

Records with only Traffic Violation charges: 10 for 3.5%

Records with only Drug Possession charges: 21 for 7.3%

Records with only Interference charges: 5 for 1.7%

Records with only "None listed" charges: 16 for 5.5%

